In [3]:
import pandas as pd
import numpy as np
from scipy.io import arff
import plotly.express as px
import plotly.graph_objects as go
import tensorflow as tf
import tensorflow_datasets as tfds
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

In [4]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [69]:
data = arff.loadarff('DryBeanDataset/Dry_Bean_Dataset.arff')
df = pd.DataFrame(data[0])

df['Class'] = df['Class'].str.decode('utf-8')
le = preprocessing.LabelEncoder()
le.fit(df['Class'])
df['Class'] = le.transform(df['Class'])
df.head()

X = tf.convert_to_tensor(df.iloc[:, :-1])

X = tf.divide(
   tf.subtract(
      X, 
      tf.reduce_min(X)
   ), 
   tf.subtract(
      tf.reduce_max(X), 
      tf.reduce_min(X)
   )
)

Y = df.pop('Class')
Y = tf.keras.utils.to_categorical(Y)

In [72]:
Y.shape

(13611, 7)

In [81]:
model = tf.keras.models.Sequential([
    tf.keras.Input(shape=(16,)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(128, activation='sigmoid'),
    tf.keras.layers.Dense(128, activation='sigmoid'),
    tf.keras.layers.Dense(128, activation='sigmoid'),
    tf.keras.layers.Dense(7),
    tf.keras.layers.Softmax(),
])

loss_fn = tf.keras.losses.CategoricalCrossentropy(from_logits=False)
model.compile(optimizer='adam',
                loss=loss_fn,
                metrics=['accuracy', tf.keras.metrics.CategoricalAccuracy()])
model.fit(X, Y, epochs=20, validation_split=0.25)


Epoch 1/20
319/319 [==============================] - 1s 3ms/step - loss: 1.5605 - accuracy: 0.3293 - categorical_accuracy: 0.3293 - val_loss: 3.6567 - val_accuracy: 0.0000e+00 - val_categorical_accuracy: 0.0000e+00
Epoch 2/20
319/319 [==============================] - 1s 2ms/step - loss: 1.0008 - accuracy: 0.5730 - categorical_accuracy: 0.5730 - val_loss: 3.0751 - val_accuracy: 0.0000e+00 - val_categorical_accuracy: 0.0000e+00
Epoch 3/20
319/319 [==============================] - 1s 2ms/step - loss: 0.9157 - accuracy: 0.6067 - categorical_accuracy: 0.6067 - val_loss: 2.9575 - val_accuracy: 0.0000e+00 - val_categorical_accuracy: 0.0000e+00
Epoch 4/20
319/319 [==============================] - 1s 2ms/step - loss: 0.8903 - accuracy: 0.6118 - categorical_accuracy: 0.6118 - val_loss: 2.1725 - val_accuracy: 0.0000e+00 - val_categorical_accuracy: 0.0000e+00
Epoch 5/20
319/319 [==============================] - 1s 2ms/step - loss: 0.8675 - accuracy: 0.6156 - categorical_accuracy: 0.6156 - val

In [74]:
model(X[:1])

<tf.Tensor: shape=(1, 7), dtype=float32, numpy=
array([[7.4282092e-05, 9.2925373e-10, 8.1542976e-06, 6.4450622e-01,
        7.3185684e-03, 3.3198979e-01, 1.6102953e-02]], dtype=float32)>

In [75]:
Y[:1]

array([[0., 0., 0., 0., 0., 1., 0.]], dtype=float32)

In [82]:
from sklearn.metrics import classification_report

In [96]:
Y_actual_report = np.argmax(Y, axis=1)
Y_actual_report

array([5, 5, 5, ..., 3, 3, 3], dtype=int64)

In [97]:
Y_train_actual = np.argmax(model(X), axis=1)
Y_train_actual

array([5, 5, 5, ..., 6, 6, 6], dtype=int64)

In [98]:
print(classification_report(Y_actual_report, Y_train_actual))

              precision    recall  f1-score   support

           0       0.41      0.31      0.35      1322
           1       1.00      1.00      1.00       522
           2       0.64      0.69      0.66      1630
           3       1.00      0.11      0.20      3546
           4       0.61      0.59      0.60      1928
           5       0.24      0.51      0.33      2027
           6       0.53      0.78      0.63      2636

    accuracy                           0.49     13611
   macro avg       0.63      0.57      0.54     13611
weighted avg       0.64      0.49      0.46     13611



In [99]:
from sklearn.decomposition import PCA


In [103]:
pca = PCA(whiten=True)
X_pca = pca.fit_transform(X)


In [104]:
X_pca

array([[-0.8410975 , -0.09551296,  1.43647106, ..., -0.73509804,
        -1.17608335,  3.92515214],
       [-0.82761116,  0.2533534 ,  1.26276692, ...,  2.99772966,
        -1.442491  ,  5.66770696],
       [-0.80793456, -0.16863269,  1.28636774, ..., -1.06671131,
        -1.05034499,  3.29322129],
       ...,
       [-0.37411608, -0.37725903,  0.21579807, ...,  0.61223569,
         0.91750653, -0.20285242],
       [-0.3723104 , -0.10392036,  0.23797653, ...,  0.86076833,
         0.99780453, -0.24872334],
       [-0.37324983, -0.33834346, -0.07918126, ..., -0.37700784,
         0.88230648,  0.33593882]])

In [105]:
model.fit(X_pca, Y, epochs=20, validation_split=0.25)

Epoch 1/20
319/319 [==============================] - 1s 4ms/step - loss: 1.2825 - accuracy: 0.6464 - categorical_accuracy: 0.6464 - val_loss: 6.2618 - val_accuracy: 0.0000e+00 - val_categorical_accuracy: 0.0000e+00
Epoch 2/20
319/319 [==============================] - 1s 3ms/step - loss: 0.4990 - accuracy: 0.7967 - categorical_accuracy: 0.7967 - val_loss: 6.2512 - val_accuracy: 0.0021 - val_categorical_accuracy: 0.0021
Epoch 3/20
319/319 [==============================] - 1s 3ms/step - loss: 0.3761 - accuracy: 0.8645 - categorical_accuracy: 0.8645 - val_loss: 6.4080 - val_accuracy: 0.1693 - val_categorical_accuracy: 0.1693
Epoch 4/20
319/319 [==============================] - 1s 3ms/step - loss: 0.2492 - accuracy: 0.9417 - categorical_accuracy: 0.9417 - val_loss: 6.8364 - val_accuracy: 0.2468 - val_categorical_accuracy: 0.2468
Epoch 5/20
319/319 [==============================] - 1s 3ms/step - loss: 0.2067 - accuracy: 0.9451 - categorical_accuracy: 0.9451 - val_loss: 7.5550 - val_accu